In [ ]:
import numpy as np
import pandas as pd

import holoviews as hv
import datashader
from holoviews.operation.datashader import aggregate, shade, datashade, dynspread
from holoviews.operation import decimate

import strax

import gc
real_gc_collect = gc.collect

In [ ]:
# Get ADC->pe multiplicative conversion factor
from pax.configuration import load_configuration
from pax.dsputils import adc_to_pe
pax_config = load_configuration('XENON1T')["DEFAULT"]
tpc_r = pax_config['tpc_radius']

to_pe = np.array([adc_to_pe(pax_config, ch) 
                  for ch in range(pax_config['n_channels'])])

# Get locations of PMTs
r = []
for q in pax_config['pmts']:
    r.append(dict(x=q['position']['x'],
                  y=q['position']['y'],
                  i=q['pmt_position'],
                  array=q.get('array', 'other')))
f = 1.08
pmt_locs = pd.DataFrame(r)
n_top = len(pax_config['channels_top'])

In [ ]:
# Load and baseline data
records = strax.load('tl_z')
strax.process.baseline(records)
records = strax.process.sort_by_time(records)

In [ ]:
n_coin = coincidence_level(records, 100)

In [ ]:
n_coin.max()

In [ ]:
#i = np.argmax(n_coin)

In [ ]:
#np.diff(records['time'][i - 200:i + 300])

In [ ]:
(records['record_i'] > 0).sum() / len(records)

In [ ]:
np.histogram(records['channel'], bins=np.arange(0, 260) - 0.5)[0].max() / len(records)

In [ ]:
(n_coin == 1).sum() / len(records)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.hist(n_coin, np.arange(0, 100)-0.5);

In [ ]:
# Somebody thought it was a good idea to call gc.collect explicitly somewhere in holoviews
# This makes dynamic PMT maps super slow
# Until I trace the offender:
gc.collect = lambda : None

In [ ]:
records = records[records['channel'] != 87]

In [ ]:
# Integral in pe
areas = records['data'].sum(axis=1) * to_pe[records['channel']]

# Convert to pandas dataframe
times = records['time']
times = (times - times[0])/1e9
df = pd.DataFrame(dict(area=areas,
                       time=times, 
                       channel=records['channel']))

# Convert to holoviews Points
points = hv.Points(df, 
                   kdims=[hv.Dimension('time', label='Time', unit='sec' ),
                          hv.Dimension('channel', label='PMT number', range=(0, 260))], 
                   vdims=[hv.Dimension('area', label='Area', unit='pe', 
                                       #range=(0, 1000)
                                      )])

In [ ]:
%opts Points.PMTPattern [color_index=2 tools=['hover'] show_grid=False] (size=17)

def pattern_plot(array, areas):
    mask = pmt_locs['array'] == array
    d = pmt_locs[mask].copy()
    d['area'] = areas[mask]
    d = hv.Dataset(d,  
                   kdims=[hv.Dimension('x', unit='cm', range=(-tpc_r * f, tpc_r * f)),
                          hv.Dimension('y', unit='cm', range=(-tpc_r * f, tpc_r * f)),
                          hv.Dimension('i', label='PMT number'),
                          hv.Dimension('area', label='Area', unit='PE')])
    return d.to(hv.Points, vdims=['area', 'i'], group='PMTPattern', label=array.capitalize())

def areas_between(t_0, t_1):
    mask = (t_0 <= points['time']) & (points['time'] < t_1)
    ps = points[mask]
    areas = np.bincount(ps['channel'], weights=ps['area'], minlength=len(pmt_locs))
    return areas

def pmt_map(t_0, t_1, array='top'):
    areas = areas_between(t_0, t_1)
    return pattern_plot(array, areas)

In [ ]:
hv.extension('bokeh')

In [ ]:
hv.DynamicMap(pmt_map, kdims=['t_0', 't_1']).redim.range(t_0=(0., 1.), t_1=(0., 1.))

In [ ]:
"""
%%opts QuadMesh [width=n_bins_t height=400 tools=['xbox_select']] (alpha=0 hover_line_alpha=1 hover_fill_alpha=0)

def selected_pmt_area(index):
    # NB: If you make exceptions in these callbacks, you get nothing!    
    # You also can't print anything. What kind of ghostly process is this running in??
    
    selected_bins = index
    # Get displayed time range
    if xyrange.x_range is None:
        t_0 = times[0]
        t_1 = times[-1]
    else:
        t_0 = xyrange.x_range[0]
        t_1 = xyrange.x_range[1]
    t_range = t_1 - t_0

    if len(selected_bins):
        tsel_0 = t_0 + selected_bins[0] * t_range/n_bins_t
        tsel_1 = t_0 + selected_bins[-1] * t_range/n_bins_t

        #return pmt_maps(tsel_0, tsel_1)
    
    return pmt_maps(t_0, t_1)

selection = hv.streams.Selection1D(source=points)  

quadmesh_helper = aggregate(points, width=40, height=20, 
                            streams=[xyrange, selection]).map(hv.QuadMesh, hv.Image)
                            
                            
tools=[wzt, 'xbox_select']
""";

In [ ]:
# xyrange = hv.streams.RangeXY(source=points)
# pulse_image = datashade(points)

# layout = pulse_image * quadmesh_helper + hv.DynamicMap(selected_pmt_area, streams=[selection]).collate()
# layout.cols(1)

In [ ]:
# Custom wheel zoom tool that only zooms in time
from bokeh.models import WheelZoomTool
wzt = WheelZoomTool(dimensions='width')
n_bins_t = 600

Link color scales!

In [ ]:
np.argmax(areas_between(*xrange.x_range))

In [ ]:
make_pmt_map()

In [ ]:
%%opts RGB [width=600, tools=[wzt], show_grid=True, default_tools=['save', 'pan', 'box_zoom', 'save', 'reset']]
%%opts Curve [width=600, tools=[wzt], show_grid=True, default_tools=['save', 'pan', 'box_zoom', 'save', 'reset']] {+framewise}

xrange = hv.streams.RangeX(source=points)

# Datashader doesn't do 1d histograms. 
# It can compute a 2d histogram and then sum it though...
# See https://github.com/bokeh/datashader/issues/225
agg = aggregate(points, 
                aggregator=datashader.sum('area'), 
                streams=[xrange], 
                x_sampling=1e-8, 
                height=2)
waveform = agg.map(lambda x: x.reduce(channel=np.sum), hv.Image)

channelmap = dynspread(datashade(points,
                                 streams=[xrange], 
                                 y_range=(0, 260)))

def make_pmt_map(x_range, array='top'):
    if x_range is None:
        return pmt_map(times[0], times[-1], array=array)
    return pmt_map(*x_range, array=array)

from functools import partial
top_map = hv.DynamicMap(partial(make_pmt_map, array='top'), streams=[xrange])
bot_map = hv.DynamicMap(partial(make_pmt_map, array='bottom'), streams=[xrange])

layout = waveform + top_map + channelmap + bot_map
layout.cols(2)